### This notebook is further processing submission for the hackathon 

https://datahack.analyticsvidhya.com/contest/amexpert-decipher-women-machine-learning-hackathon/

My public and private leaderboard scores are 132.4390534342 and 132.8241022151 respectively

In [115]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',100)
import warnings
warnings.filterwarnings('ignore')

In [93]:
from statistics import mode 
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error

In [36]:
train = pd.read_csv('/Users/lavanyamk/Documents/decipHER/train_bFQbE3f/train.csv')
train_dict = pd.read_excel('/Users/lavanyamk/Documents/decipHER/train_bFQbE3f/Data_Dictionary.xlsx')

In [37]:
test = pd.read_csv('/Users/lavanyamk/Documents/decipHER/test_9K3DBWQ.csv')

In [38]:
train.head()

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
0,23334,saving,M,63,305,7695.00,NaN,479.00,NaN,622.0,NaN,2.0,1.0,3.0,NaN,NaN,NaN,150000.0,NaN,NaN,NaN,NaN,NaN,118845.2,50000.0,NaN,51205.47,50259.0,3.0,1.0,50000.0,24740.22,114252.0,3.0,2.0,45000.0,20215.85,1279.37,7.0,3.0,470.0,NaN,0.00,435
1,20329,current,M,33,326,52702.00,988.72,29873.00,NaN,1087.0,2806.24,11.0,14.0,11.0,1.0,NaN,6.0,176000.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,159050.76,169812.0,20.0,9.0,70000.0,103102.85,174606.0,6.0,10.0,150000.0,122037.66,55390.00,8.0,19.0,27000.0,NaN,4963.33,1921
2,17679,current,M,44,264,10800.49,3215.52,21659.64,1607.0,1168.0,355.00,6.0,9.0,6.0,3.0,2.0,14.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51484.55,38942.0,16.0,4.0,11700.0,15283.00,23051.0,2.0,7.0,13800.0,46827.00,29297.91,6.0,19.0,20800.0,NaN,0.00,1970
3,16428,saving,M,44,233,20936.00,NaN,2094.00,NaN,11778.5,NaN,9.0,6.0,6.0,NaN,NaN,NaN,600000.0,NaN,NaN,NaN,NaN,NaN,132442.0,NaN,NaN,1981.07,2294.0,2.0,1.0,2000.0,19361.24,19529.0,1.0,1.0,19300.0,5426.55,7544.00,3.0,1.0,5300.0,NaN,1238.18,21579
4,18721,saving,F,35,324,1183.83,NaN,3656.29,NaN,945.0,NaN,2.0,7.0,9.0,NaN,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6462.00,10271.0,6.0,1.0,10000.0,7461.00,3194.0,2.0,3.0,3000.0,26178.05,53483.00,4.0,4.0,45000.0,NaN,0.00,1814


In [170]:
test.shape

(14067, 43)

In [40]:
train.shape

(32820, 44)

In [15]:
train.card_lim.isna().value_counts()

False    32811
True         9
Name: card_lim, dtype: int64

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067 entries, 0 to 14066
Data columns (total 43 columns):
id                       14067 non-null int64
account_type             14067 non-null object
gender                   14067 non-null object
age                      14067 non-null int64
region_code              14067 non-null int64
cc_cons_apr              14067 non-null float64
dc_cons_apr              5837 non-null float64
cc_cons_may              14067 non-null float64
dc_cons_may              6692 non-null float64
cc_cons_jun              14067 non-null float64
dc_cons_jun              5003 non-null float64
cc_count_apr             13029 non-null float64
cc_count_may             13525 non-null float64
cc_count_jun             13433 non-null float64
dc_count_apr             5837 non-null float64
dc_count_may             6692 non-null float64
dc_count_jun             7515 non-null float64
card_lim                 14064 non-null float64
personal_loan_active     1134 non-null fl

In [133]:
train_temp = train[['age','gender','cc_cons_apr','cc_cons_may','cc_cons_jun','cc_cons',
       'investment_1','investment_2','investment_3','investment_4','emi_active']].fillna(0)

In [134]:
gender = pd.get_dummies(train_temp.gender)

In [135]:
train_df = pd.concat([train_temp,gender[['F']]],axis=1).drop(['gender','cc_cons'],axis=1)
target_df= train[['cc_cons']]

In [104]:
#train['card_lim'].fillna(mode(train['card_lim']))

100000.0

In [91]:
import lightgbm as lgbm

In [338]:
from sklearn.linear_model import LinearRegression

In [137]:
lgbmr = lgbm.LGBMRegressor(n_estimators=500)

In [217]:
def rmsle_cv(model,df_train,df_target):
    kf = KFold(n_splits=5, shuffle=True, random_state=6).get_n_splits(df_train.values)
    rmse= np.sqrt(-cross_val_score(model, df_train.values, df_target.values, scoring="neg_mean_squared_error", cv = kf))
    print(rmse)
    return(rmse)

In [339]:
def seperate_by_age(train, test):
    pred_df=pd.DataFrame()
    ages = [20, 35, 50, 71, 250]
    for i in range(len(ages) - 1):
        globals()['train_%s' % i] = train[(train.age > ages[i]) & (train.age <= ages[i + 1])]
        train_temp = globals()['train_%s' % i][
           ['gender', 'cc_cons_apr', 'dc_cons_apr', 'cc_cons_may',
              'dc_cons_may', 'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr',
              'cc_count_may', 'cc_count_jun', 'dc_count_apr', 'dc_count_may',
              'dc_count_jun', 'investment_1', 'investment_2', 'investment_3',
              'investment_4', 'debit_amount_apr', 'credit_amount_apr',
              'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
              'debit_amount_may', 'credit_amount_may', 'credit_count_may',
              'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
              'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
              'max_credit_amount_jun', 'personal_loan_active', 'vehicle_loan_active',
              'personal_loan_closed', 'vehicle_loan_closed', 'emi_active'
           ]
        ].fillna(0)
        train_temp['card_lim'] = globals()['train_%s' % i]['card_lim'].fillna(mode(globals()['train_%s' % i]['card_lim']))
        globals()['target_%s' % i] = globals()['train_%s' % i][
           ['cc_cons']
        ]
        gender = pd.get_dummies(train_temp.gender)
        train_df = pd.concat([train_temp, gender[['M']]], axis = 1).drop(['gender'], axis = 1)
        lgbmr = lgbm.LGBMRegressor(n_estimators = 50,learning_rate=0.0)
        lr = LinearRegression()
        rmsle_cv(lgbmr, train_df, globals()['target_%s' % i])
        rmsle_cv(lr, train_df, globals()['target_%s' % i])
        globals()['train_%s' % i] = train[(train.age > ages[i]) & (train.age <= ages[i + 1])]
        lgbmr.fit(train_df, globals()['target_%s' % i])
        globals()['test_%s' % i] = test[(test.age > ages[i]) & (test.age <= ages[i + 1])]
#         globals()['test_id_%s' % i] = globals()['test_%s' % i].id
#         globals()['test_index_%s' % i] = globals()['test_%s' % i].index
        test_temp = globals()['test_%s' % i][
           ['gender', 'cc_cons_apr', 'dc_cons_apr', 'cc_cons_may',
              'dc_cons_may', 'cc_cons_jun', 'dc_cons_jun', 'cc_count_apr',
              'cc_count_may', 'cc_count_jun', 'dc_count_apr', 'dc_count_may',
              'dc_count_jun', 'investment_1', 'investment_2', 'investment_3',
              'investment_4', 'debit_amount_apr', 'credit_amount_apr',
              'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
              'debit_amount_may', 'credit_amount_may', 'credit_count_may',
              'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
              'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
              'max_credit_amount_jun', 'personal_loan_active', 'vehicle_loan_active',
              'personal_loan_closed', 'vehicle_loan_closed', 'emi_active'
           ]
        ].fillna(0)
        test_temp['card_lim'] = globals()['test_%s' % i]['card_lim'].fillna(mode(globals()['test_%s' % i]['card_lim']))
        gender_ = pd.get_dummies(test_temp.gender)
        test_df = pd.concat([test_temp, gender_[['M']]], axis = 1).drop(['gender'], axis = 1)
        predict_df = pd.DataFrame({'cc_cons':lgbmr.predict(test_df)},index=test_temp.index)
        pred_df = pd.concat([pred_df,pd.concat([globals()['test_%s' % i][['id']],predict_df],axis=1)])
        #print(list(globals()['test_index_%s' % i]))
        #pred_df.loc[list(globals()['test_index_%s' % i]), 'id'] = globals()['test_id_%s' % i]
        #pred_df.loc[list(globals()['test_index_%s' % i]), 'cc_cons'] = globals()['pred_%s' % i]
    return pred_df

In [343]:
def seperate_by_age(train, test):
    pred_df=pd.DataFrame()
    ages = [20, 35, 50, 71, 250]
    n_estimators = [50,50,150,150,150,130,100,150,150,55,55]
    for i in range(len(ages) - 1):
        globals()['train_%s' % i] = train[(train.age > ages[i]) & (train.age <= ages[i + 1])]
        train_temp = globals()['train_%s' % i][
           ['gender', 'cc_cons_may','dc_cons_may', 'cc_cons_jun', 'dc_cons_jun',
              'cc_count_may', 'cc_count_jun', 'dc_count_may',
              'dc_count_jun', 'investment_1', 'investment_2', 'investment_3',
              'investment_4','debit_amount_may', 'credit_amount_may', 'credit_count_may',
              'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
              'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
              'max_credit_amount_jun', 'personal_loan_active', 'vehicle_loan_active',
              'personal_loan_closed', 'vehicle_loan_closed', 'emi_active'
           ]
        ].fillna(0)
        print(len(train_temp))
        train_temp['card_lim'] = globals()['train_%s' % i]['card_lim'].fillna(mode(globals()['train_%s' % i]['card_lim']))
        
        globals()['target_%s' % i] = globals()['train_%s' % i][
           ['cc_cons']
        ]
        gender = pd.get_dummies(train_temp.gender)
        train_df = pd.concat([train_temp, gender[['M']]], axis = 1).drop(['gender'], axis = 1)
        lr = LinearRegression()
        rmsle_cv(lr, train_df, globals()['target_%s' % i])
        lgbmr = lgbm.LGBMRegressor(n_estimators = n_estimators[i],learning_rate=0.05)
        rmsle_cv(lgbmr, train_df, globals()['target_%s' % i])
        lgbmr.fit(train_df, globals()['target_%s' % i])
        globals()['test_%s' % i] = test[(test.age > ages[i]) & (test.age <= ages[i + 1])]
#         globals()['test_id_%s' % i] = globals()['test_%s' % i].id
#         globals()['test_index_%s' % i] = globals()['test_%s' % i].index
        test_temp = globals()['test_%s' % i][
           ['gender','cc_cons_may','dc_cons_may', 'cc_cons_jun', 'dc_cons_jun',
              'cc_count_may', 'cc_count_jun',  'dc_count_may',
              'dc_count_jun', 'investment_1', 'investment_2', 'investment_3',
              'investment_4','debit_amount_may', 'credit_amount_may', 'credit_count_may',
              'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
              'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
              'max_credit_amount_jun', 'personal_loan_active', 'vehicle_loan_active',
              'personal_loan_closed', 'vehicle_loan_closed', 'emi_active'
           ]
        ].fillna(0)
        test_temp['card_lim'] = globals()['test_%s' % i]['card_lim'].fillna(mode(globals()['test_%s' % i]['card_lim']))
        gender_ = pd.get_dummies(test_temp.gender)
        test_df = pd.concat([test_temp, gender_[['M']]], axis = 1).drop(['gender'], axis = 1)
        predict_df = pd.DataFrame({'cc_cons':lgbmr.predict(test_df)},index=test_temp.index)
        pred_df = pd.concat([pred_df,pd.concat([globals()['test_%s' % i][['id']],predict_df],axis=1)])
        #print(list(globals()['test_index_%s' % i]))
        #pred_df.loc[list(globals()['test_index_%s' % i]), 'id'] = globals()['test_id_%s' % i]
        #pred_df.loc[list(globals()['test_index_%s' % i]), 'cc_cons'] = globals()['pred_%s' % i]
    return pred_df


In [344]:
pred = seperate_by_age(train,test)

18567
[ 9091.16215855  8246.9941482  10231.23602728  8330.56996121
  9523.11941062]
[ 9010.86291823  8316.674315   10096.95212333  8629.59895335
  9541.96764972]
12493
[15763.05659796 12461.54955298 13540.2011588  12214.04622689
 11306.68083145]
[16429.67584718 12261.75276271 13825.73743121 12426.68307397
 11143.76977102]
1735
[15083.95741088 17682.55035117 13519.7517493  22852.93635878
 66792.58371932]
[15779.31017022 18617.64839523 14679.47993709 15175.62852706
 15455.8163757 ]
25
[58884.08688287 13747.71349502 23589.07368507 19379.45245872
 12554.79327878]
[ 3958.53484543 10990.7243575  12027.16674907  4085.07310216
  5969.97139775]


In [330]:
#pred.loc[pred.cc_cons<0,'cc_cons'] = -(pred[pred.cc_cons<0])
pred[pred.cc_cons<0]

,id,cc_cons
8569,27798,-6174.821923
10932,2227,-3796.635882
13690,5256,-1457.974358


In [233]:
pred.loc[pred.id==44472,'cc_cons']=2763.6

In [230]:
pred.loc[pred.id==249,'cc_cons']=1957.39

In [327]:
pred.sort_index().to_csv('Submission_5_seperated.csv',index=False)

In [139]:
rmsle_cv(lgbmr,train_df,target_df)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[12747.39154628 11181.21821074 11972.1879408  11022.85917974
 11461.28217755]


array([12747.39154628, 11181.21821074, 11972.1879408 , 11022.85917974,
       11461.28217755])

In [62]:
train.columns

Index(['id', 'account_type', 'gender', 'age', 'region_code', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim',
       'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'loan_enq', 'emi_active', 'cc_cons'],
      dtype='object')

In [144]:
lgbmr.fit(train_df,target_df)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [145]:
train.region_code.nunique()

413

In [146]:
def preprocess_test(test_,lgbmreg):
    test_temp = test_[['age','gender', 'cc_cons_apr','dc_cons_apr', 'cc_cons_may',
                       'dc_cons_may', 'cc_cons_jun','dc_cons_jun', 'cc_count_apr',
                       'cc_count_may', 'cc_count_jun','dc_count_apr', 'dc_count_may',
                       'dc_count_jun''investment_1','investment_2','investment_3',
                       'investment_4','personal_loan_active','vehicle_loan_active',
                       'personal_loan_closed','vehicle_loan_closed','emi_active']].fillna(0)
    gender = pd.get_dummies(test_temp.gender)
    test_df = pd.concat([test_temp,gender[['F']]],axis=1).drop(['gender'],axis=1)
    return lgbmreg.predict(test_df)

In [ ]:
id_

In [87]:
# ages = [0,30,50,71,250]
# for i in range(len(ages)-1):
#     globals()['train_%s' % i] = train[(train.age>ages[i]) & (train.age<=ages[i+1])]
#     globals()['id_%s' % i] = globals()['train_%s' % i].index

In [82]:
len(train_0)+len(train_1)+len(train_2)+len(train_3)

32820

In [80]:
print(len(train[(train.age>71)]))
print(len(train[(train.age>50) & (train.age<=71)]))
print(len(train[(train.age<=50) & (train.age>30)]))
print(len(train[train.age<=30]))

25
1735
22958
8102


In [76]:
train[(train.age>70)]['age']

1121     119
2496     221
2914     119
3205     222
4313      71
7071     223
7823     223
8405      71
9541     123
11455    118
13862    219
14967    121
15736    120
16053    118
16844    223
17295    124
17934    224
21594    121
22271    220
23789    221
25762    220
26013    221
26544    121
28004    124
28166    219
29808    124
31130    120
Name: age, dtype: int64

In [51]:
train[train.age<=30].columns

Index(['id', 'account_type', 'gender', 'age', 'region_code', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim',
       'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'loan_enq', 'emi_active', 'cc_cons'],
      dtype='object')

In [47]:
#train_30 = train[train.age<=30]
#train_50 = train[(train.age<=50) & (train.age>30)]
#train_70 = train[(train.age>50) & (train.age<=70)]
#train_70more = train[train.age>70]
len(train[train.age>70]) + len(train[(train.age>50) & (train.age<=70)]) + len(train[(train.age<=50) & (train.age>30)]) +len(train[train.age<=30])


32820

In [48]:
train.shape

(32820, 44)

In [61]:
train.loc[train.card_lim.isna()][['cc_cons_apr','cc_cons_may']].apply(lambda x:max(x))

cc_cons_apr    48358.925
cc_cons_may    91994.850
dtype: float64

In [11]:
test[test.card_lim.isna()]

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
850,7194,current,M,35,274,3018.02,160.0,2304.00,NaN,809.04,NaN,NaN,1.0,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58998.18,58873.80,10.0,3.0,33155.0,35329.88,39287.0,2.0,6.0,33113.0,89777.25,89888.0,5.0,15.0,46300.0,NaN,0.00
5822,10145,saving,M,33,349,39926.98,NaN,5438.10,NaN,1866.00,NaN,7.0,3.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110000.0,NaN,NaN,54654.00,35532.00,8.0,1.0,35270.0,128104.10,62709.0,3.0,22.0,47622.0,52918.67,43895.0,2.0,17.0,42789.0,NaN,0.00
9948,29307,current,M,42,507,6894.24,NaN,12846.93,NaN,15682.00,NaN,9.0,12.0,34.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9245.0,NaN,30484.41,0.0,2.0,30350.0,36634.41,33529.0,2.0,5.0,27295.0,68510.50,41430.0,4.0,7.0,30350.0,NaN,2027.59


In [23]:
(train.columns

Index(['id', 'account_type', 'gender', 'age', 'region_code', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim',
       'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'loan_enq', 'emi_active', 'cc_cons'],
      dtype='object')

In [75]:
test[test.card_lim<=1]

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
2533,32707,current,M,30,548,870.000,NaN,620.0,NaN,10589.79,NaN,1.0,1.0,7.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3075.25,5428.0,3.0,2.0,4000.0,7366.88,93115.0,2.0,4.0,85000.0,10438.88,468.00,1.0,4.0,422.0,NaN,0.0
6589,24168,saving,M,29,507,5551.840,NaN,22047.5,NaN,563.00,NaN,4.0,12.0,15.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6297.00,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,581.00,3256.00,2.0,1.0,3000.0,NaN,0.0
8803,16641,saving,M,44,307,727.005,NaN,868.0,NaN,281.01,310.0,NaN,1.0,4.0,NaN,NaN,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.89,30199.0,3.0,1.0,30000.0,386.00,NaN,0.0,2.0,NaN,111568.75,67215.25,4.0,6.0,66500.0,NaN,0.0


In [62]:
train[train.card_lim==1]

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
10313,42897,saving,M,52,948,1931.420,NaN,1370.00,NaN,2128.71,NaN,5.0,4.0,7.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,722.75,5187.0,3.0,2.0,3255.0,25675.00,6057.0,2.0,7.0,5391.0,NaN,14037.0,4.0,0.0,8081.0,NaN,0.00,990
14369,35138,current,M,35,233,1327.000,NaN,2582.40,5448.0,2568.20,NaN,8.0,15.0,NaN,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10102.0,0.0,1.0,10000.0,10884.50,NaN,0.0,9.0,NaN,3743.50,417.0,1.0,4.0,189.0,NaN,5895.10,2
17793,45357,current,F,28,706,15051.540,NaN,27214.11,NaN,66377.00,NaN,6.0,7.0,18.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,47835
25126,46300,saving,F,52,565,2903.085,602.0,3285.17,166.0,577.00,NaN,NaN,4.0,9.0,5.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,541.00,NaN,5.0,0.0,NaN,145758.75,119518.0,4.0,14.0,49000.0,25204.25,24062.0,2.0,16.0,23807.0,NaN,0.00,647
28204,24445,current,M,42,407,6762.790,NaN,4052.50,NaN,1049.00,NaN,2.0,3.0,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3466.25,10417


In [67]:
train[train.card_lim==1][['cc_cons_apr','cc_cons_may','cc_cons_jun']]

,cc_cons_apr,cc_cons_may,cc_cons_jun
10313,1931.420,1370.00,2128.71
14369,1327.000,2582.40,2568.20
17793,15051.540,27214.11,66377.00
25126,2903.085,3285.17,577.00
28204,6762.790,4052.50,1049.00


In [84]:
train[['age','gender','region_code','cc_cons_may','cc_cons_jun','cc_cons',
       'investment_1','investment_2','investment_3','investment_4','emi_active']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32820 entries, 0 to 32819
Data columns (total 12 columns):
age             32820 non-null int64
gender          32820 non-null object
region_code     32820 non-null int64
cc_cons_apr     32820 non-null float64
cc_cons_may     32820 non-null float64
cc_cons_jun     32820 non-null float64
cc_cons         32820 non-null int64
investment_1    1426 non-null float64
investment_2    2338 non-null float64
investment_3    1332 non-null float64
investment_4    406 non-null float64
emi_active      32820 non-null float64
dtypes: float64(8), int64(3), object(1)
memory usage: 3.0+ MB


In [166]:
submission1 = pd.DataFrame({'id':test.id,'cc_cons':preprocess_test(test,lgbmr)})
#.to_csv('Submission1_10_features.csv',index=False)

In [169]:
submission1.loc[submission1.cc_cons<0,'cc_cons']=-(submission1[submission1.cc_cons<0])

In [170]:
submission1.to_csv('Submission1_10_features.csv',index=False)

In [171]:
submission1.loc[submission1.cc_cons<0]

,id,cc_cons
